# Mod Spec

In [5]:
import scipy.io
import glob
import numpy as np
mod_path = '../../data/output_mats\\'

In [2]:
def load_mat(filename):
    mat = scipy.io.loadmat( mod_path +  filename )
    #mat is a dict with following keys dict_keys(['__header__', '__version__', '__globals__', 'filename'])
    print(filename + " file loaded")
    return mat[filename]

In [8]:
modBoatTrain = load_mat('modspec_boat_train')
modBoatTest = load_mat('modspec_boat_test')
modHeliTrain = load_mat('modspec_heli_train')
modHeliTest = load_mat('modspec_heli_test')

modspec_boat_train file loaded
modspec_boat_test file loaded
modspec_heli_train file loaded
modspec_heli_test file loaded


In [9]:
trainX = np.concatenate((modBoatTrain,modHeliTrain),axis=0)
trainY = np.concatenate((np.ones((modBoatTrain.shape[0],1)) , np.zeros((modHeliTrain.shape[0],1))),axis=0)
testX = np.concatenate((modBoatTest,modHeliTest),axis=0)
testY = np.concatenate((np.ones((modBoatTest.shape[0],1)) , np.zeros((modHeliTest.shape[0],1))),axis=0)

In [10]:
trainX.shape, trainY.shape, testX.shape, testY.shape

((155853, 544), (155853, 1), (157288, 544), (157288, 1))

In [11]:
from sklearn.linear_model import LogisticRegression

In [16]:
import timeit

start = timeit.timeit()

clf = LogisticRegression(penalty='l2')
clf.fit(trainX,trainY)
print(clf.score(trainX,trainY), clf.score(testX, testY))

end = timeit.timeit()
print(end - start)


C:\Anaconda_3_5\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.738914233284 0.443791007578
0.005299099334479251


In [14]:
pickle sklearn.externals import joblib

In [15]:
joblib.dump(clf, mod_path + 'ModSpec_LR_L1.pkl') 

['../../data/output_mats\\ModSpec_LR_L1.pkl']

# GMM

In [19]:
from sklearn.mixture import GaussianMixture as GMM

train_boat =  trainX[trainY.ravel()==1]  #X_boat[:-5120]
train_heli = trainX[trainY.ravel()==0]
test_boat = testX[testY.ravel()==1]
test_heli = testX[testY.ravel()==0]

gmm = {'boat':None, 'heli':None}

gmm["boat"] = GMM(n_components=12)
gmm["boat"].fit(train_boat)

gmm["heli"] = GMM(n_components=12)
gmm["heli"].fit(train_heli)

C:\Anaconda_3_5\lib\site-packages\sklearn\mixture\base.py:237: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)


GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=100,
        means_init=None, n_components=12, n_init=1, precisions_init=None,
        random_state=None, reg_covar=1e-06, tol=0.001, verbose=0,
        verbose_interval=10, warm_start=False, weights_init=None)

In [20]:
scores = np.zeros((trainY.shape[0], 2))

scores[:, 0] = gmm["boat"].score_samples(trainX)
scores[:, 1] = gmm["heli"].score_samples(trainX)

pred = np.greater(scores[:, 1], scores[:, 0])
error = np.not_equal(pred, np.squeeze(trainY))

error_rate = np.sum(error) / len(error)
print(error_rate)

0.720005389694
